In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

%matplotlib inline 


In [4]:
df_ACS15B25063=pd.read_csv('/users/richardkornblith/Data_Science/NYCHR/Data_for_NYCHR/ACS_15_5YR_by_tract/ACS_15_5YR_B25063_with_ann.csv')
df_ACS17B25063=pd.read_csv('/users/richardkornblith/Data_Science/NYCHR/Data_for_NYCHR/ACS_17_5YR_by_tract/ACS_17_5YR_B25063_with_ann.csv')
df_ACS14B25063=pd.read_csv('/users/richardkornblith/Data_Science/NYCHR/Data_for_NYCHR/ACS_14_5YR_by_tract/ACS_14_5YR_B25063_with_ann.csv')


df_ACS14B25063.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,...,HD01_VD20,HD02_VD20,HD01_VD21,HD02_VD21,HD01_VD22,HD02_VD22,HD01_VD23,HD02_VD23,HD01_VD24,HD02_VD24
0,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,Estimate; With cash rent:,Margin of Error; With cash rent:,Estimate; With cash rent: - Less than $100,Margin of Error; With cash rent: - Less than $100,Estimate; With cash rent: - $100 to $149,...,"Estimate; With cash rent: - $1,000 to $1,249","Margin of Error; With cash rent: - $1,000 to $...","Estimate; With cash rent: - $1,250 to $1,499","Margin of Error; With cash rent: - $1,250 to $...","Estimate; With cash rent: - $1,500 to $1,999","Margin of Error; With cash rent: - $1,500 to $...","Estimate; With cash rent: - $2,000 or more","Margin of Error; With cash rent: - $2,000 or more",Estimate; No cash rent,Margin of Error; No cash rent
1,1400000US36061000100,36061000100,"Census Tract 1, New York County, New York",0,11,0,11,0,11,0,...,0,11,0,11,0,11,0,11,0,11
2,1400000US36061000201,36061000201,"Census Tract 2.01, New York County, New York",921,76,921,76,0,11,19,...,144,74,12,21,7,12,70,40,0,11
3,1400000US36061000202,36061000202,"Census Tract 2.02, New York County, New York",2786,217,2712,231,0,16,16,...,252,156,23,38,0,16,64,81,74,79
4,1400000US36061000500,36061000500,"Census Tract 5, New York County, New York",0,11,0,11,0,11,0,...,0,11,0,11,0,11,0,11,0,11


In [5]:
df_ACS15B25063.drop([0], inplace=True)
df_ACS17B25063.drop([0], inplace=True)
df_ACS14B25063.drop([0], inplace=True)

In [9]:
df_ACS15B25063.rename(columns={'GEO.id2':'Tract','HD01_VD02':'Units','HD01_VD23':'2000-2499','HD01_VD24':'2500-2999','HD01_VD25':'3000-3499','HD01_VD26':'3500 or more'}, inplace=True)
df_ACS17B25063.rename(columns={'GEO.id2':'Tract','HD01_VD02':'Units','HD01_VD23':'2000-2499','HD01_VD24':'2500-2999','HD01_VD25':'3000-3499','HD01_VD26':'3500 or more'}, inplace=True)
df_ACS14B25063.rename(columns={'GEO.id2':'Tract','HD01_VD02':'Units','HD01_VD23':'2000 or more'}, inplace=True)
df_ACS14B25063.columns

Index(['GEO.id', 'Tract', 'GEO.display-label', 'HD01_VD01', 'HD02_VD01',
       'Units', 'HD02_VD02', 'HD01_VD03', 'HD02_VD03', 'HD01_VD04',
       'HD02_VD04', 'HD01_VD05', 'HD02_VD05', 'HD01_VD06', 'HD02_VD06',
       'HD01_VD07', 'HD02_VD07', 'HD01_VD08', 'HD02_VD08', 'HD01_VD09',
       'HD02_VD09', 'HD01_VD10', 'HD02_VD10', 'HD01_VD11', 'HD02_VD11',
       'HD01_VD12', 'HD02_VD12', 'HD01_VD13', 'HD02_VD13', 'HD01_VD14',
       'HD02_VD14', 'HD01_VD15', 'HD02_VD15', 'HD01_VD16', 'HD02_VD16',
       'HD01_VD17', 'HD02_VD17', 'HD01_VD18', 'HD02_VD18', 'HD01_VD19',
       'HD02_VD19', 'HD01_VD20', 'HD02_VD20', 'HD01_VD21', 'HD02_VD21',
       'HD01_VD22', 'HD02_VD22', '2000 or more', 'HD02_VD23', 'HD01_VD24',
       'HD02_VD24'],
      dtype='object')

In [10]:
rents2015_df = df_ACS15B25063[['Tract','Units', '2000-2499', '2500-2999','3000-3499','3500 or more']]
rents2015_df = rents2015_df.astype(int)
rents2017_df = df_ACS15B25063[['Tract','Units', '2000-2499', '2500-2999','3000-3499','3500 or more']]
rents2017_df = rents2017_df.astype(int)
rents2014_df = df_ACS15B25063[['Tract','Units','2000 or more']]
rents2014_df = rents2014_df.astype(int)



rents2014_df.info()

KeyError: "['2000 or more'] not in index"

In [30]:
rents2015_df.head(50)

,Tract,Units,2000-2499,2500-2999,3000-3499,3500 or more
1,36061000100,0,0,0,0,0
2,36061000201,972,64,11,0,0
3,36061000202,2649,0,15,11,0
4,36061000500,0,0,0,0,0
5,36061000600,4069,59,31,31,67
6,36061000700,3533,760,1226,997,420
7,36061000800,3519,77,15,20,0
8,36061000900,653,38,32,119,420
9,36061001001,174,5,0,0,0
10,36061001002,2266,0,0,0,0


In [31]:
rents2017_df.head(50)

,Tract,Units,2000-2499,2500-2999,3000-3499,3500 or more
1,36061000100,0,0,0,0,0
2,36061000201,972,64,11,0,0
3,36061000202,2649,0,15,11,0
4,36061000500,0,0,0,0,0
5,36061000600,4069,59,31,31,67
6,36061000700,3533,760,1226,997,420
7,36061000800,3519,77,15,20,0
8,36061000900,653,38,32,119,420
9,36061001001,174,5,0,0,0
10,36061001002,2266,0,0,0,0


In [28]:
rents2015_df.info()
rents2017_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 1 to 288
Data columns (total 6 columns):
Tract           288 non-null int64
Units           288 non-null int64
2000-2499       288 non-null int64
2500-2999       288 non-null int64
3000-3499       288 non-null int64
3500 or more    288 non-null int64
dtypes: int64(6)
memory usage: 15.8 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 1 to 288
Data columns (total 6 columns):
Tract           288 non-null int64
Units           288 non-null int64
2000-2499       288 non-null int64
2500-2999       288 non-null int64
3000-3499       288 non-null int64
3500 or more    288 non-null int64
dtypes: int64(6)
memory usage: 15.8 KB
